Run the following code before running this script

ROAST to get Bvals

0.5_extract_sphere_data.ipynb

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score

#
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Flatten, Dense


from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
#from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from scikeras.wrappers import KerasRegressor
from sklearn.ensemble import AdaBoostRegressor

import pickle
from keras.models import load_model

# Load prepared data

In [2]:
dir_path = "/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/"
# dir_path = "/Users/mriworkshop/Documents/TDCS/code/tdcs_thesis/"
save_path = dir_path+"data/raw/"
img_path =  dir_path+"data/processed/"
model_path = dir_path+"models/"

## fmap mean all experiments

In [ ]:
file_mean = save_path+"fmap_mean_40.txt"
columns_mean =['exp', 'mini_exp', 'i', 'j', 'k', 'mean0', 'mean1', 'mean2', 'mean3', 'mean4']
data = np.loadtxt(file_mean);

In [ ]:
df = pd.DataFrame(data, columns=columns_mean)
df = df.astype({"exp": int, "i": int, "j": int, "k": int, "mini_exp": int})
df

# Separate -ON case

In [3]:
file_mean = save_path+"fmap_mean_neg_amp_42.txt"
columns_mean =['exp', 'mini_exp', 'i', 'j', 'k', 'amp', 'neg', 'mean0', 'mean1', 'mean2','target']
data = np.loadtxt(file_mean);

In [4]:
df = pd.DataFrame(data, columns=columns_mean)
df = df.astype({"exp": int, "i": int, "j": int, "k": int, "mini_exp": int, "amp":int, "neg": int})
df

,exp,mini_exp,i,j,k,amp,neg,mean0,mean1,mean2,target
0,42,1,0,0,0,2,0,0.0,0.0,0.0,0.578396
1,42,1,0,0,1,2,0,0.0,0.0,0.0,0.584610
2,42,1,0,0,2,2,0,0.0,0.0,0.0,0.590888
3,42,1,0,0,3,2,0,0.0,0.0,0.0,0.597230
4,42,1,0,0,4,2,0,0.0,0.0,0.0,0.603638
...,...,...,...,...,...,...,...,...,...,...,...
1081339,42,3,43,63,59,2,1,0.0,0.0,0.0,1.865481
1081340,42,3,43,63,60,2,1,0.0,0.0,0.0,1.910840
1081341,42,3,43,63,61,2,1,0.0,0.0,0.0,1.957465
1081342,42,3,43,63,62,2,1,0.0,0.0,0.0,2.005388


In [5]:
df.loc[(df['i']==32)&(df['j']==30)&(df['k']==40)]

,exp,mini_exp,i,j,k,amp,neg,mean0,mean1,mean2,target
133032,42,1,32,30,40,2,0,-647.541748,-641.784790,-637.159912,1.212172
313256,42,1,32,30,40,2,1,-637.159912,-636.072449,-632.968689,1.212172
493480,42,2,32,30,40,2,0,-628.600159,-627.323364,-622.702576,1.212172
673704,42,2,32,30,40,2,1,-622.702576,-619.876587,-617.423645,1.212172
853928,42,3,32,30,40,2,0,-615.325317,-614.063293,-611.295593,1.212172
1034152,42,3,32,30,40,2,1,-611.295593,-611.259521,-609.233459,1.212172


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1081344 entries, 0 to 1081343
Data columns (total 11 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   exp       1081344 non-null  int64  
 1   mini_exp  1081344 non-null  int64  
 2   i         1081344 non-null  int64  
 3   j         1081344 non-null  int64  
 4   k         1081344 non-null  int64  
 5   amp       1081344 non-null  int64  
 6   neg       1081344 non-null  int64  
 7   mean0     1081344 non-null  float64
 8   mean1     1081344 non-null  float64
 9   mean2     1081344 non-null  float64
 10  target    1081344 non-null  float64
dtypes: float64(4), int64(7)
memory usage: 90.8 MB


# Split data

## Get one experiment

In [7]:
df_test1 = df[(df['exp']==42) & (df['mini_exp']==1) & (df['neg']==1)]
df_test2 = df[(df['exp']==42) & (df['mini_exp']==1) & (df['neg']==0)]
# df_test3 = df[(df['exp']==42) & (df['mini_exp']==3)]

In [8]:
print(df_test1.shape)
print(df_test2.shape)
# print(df_test3.shape)

(180224, 11)
(180224, 11)


## None zero

In [ ]:
# 5 attributes
df_nonzero = df[(df['mean0']!=0.0) & (df['mean1']!=0.0) & (df['mean2']!=0.0) & (df['mean3']!=0.0) & (df['mean4']!=0.0)]

In [9]:
# 3 attributes
df_nonzero = df[(df['mean0']!=0.0) & (df['mean1']!=0.0) & (df['mean2']!=0.0)]

In [10]:
df_nonzero

,exp,mini_exp,i,j,k,amp,neg,mean0,mean1,mean2,target
10074,42,1,2,29,26,2,0,-984.707153,-970.159729,-974.554504,0.798312
10076,42,1,2,29,28,2,0,-333.152374,-270.754669,-102.585724,0.819827
10137,42,1,2,30,25,2,0,-788.129700,-787.770996,-808.987488,0.786963
10138,42,1,2,30,26,2,0,-1038.672119,-1019.283020,-1018.710083,0.797582
13853,42,1,3,24,29,2,0,-352.913147,-328.458252,-345.069885,0.837726
...,...,...,...,...,...,...,...,...,...,...,...
1066912,42,3,40,30,32,2,1,531.209412,524.617249,528.361572,1.120325
1066965,42,3,40,31,21,2,1,-738.791687,-309.708313,248.529739,0.942770
1066966,42,3,40,31,22,2,1,-854.326355,-827.500061,-840.415222,0.957650
1067028,42,3,40,32,20,2,1,-524.266296,-949.864380,-932.718628,0.927170


In [38]:
df4_pred1 =  df_nonzero[(df_nonzero['exp']==42) & (df_nonzero['mini_exp']==1) & (df_nonzero['neg']==1)]
df4_pred2 =  df_nonzero[(df_nonzero['exp']==42) & (df_nonzero['mini_exp']==1) & (df_nonzero['neg']==0)]
# df4_pred3 =  df_nonzero[(df_nonzero['exp']==42) & (df_nonzero['mini_exp']==3)]

In [39]:
df4_pred1.iloc[:, 7:-1]

,mean0,mean1,mean2
190298,-974.554504,-988.312378,-973.724548
190300,-102.585724,-143.370544,-117.800835
190361,-808.987488,-804.097961,-796.817200
190362,-1018.710083,-1042.302490,-1022.633850
194077,-345.069885,-351.042358,-337.602417
...,...,...,...
346016,499.902985,502.789764,508.389282
346069,-799.488831,-320.193604,3493.769287
346070,-875.329651,-849.496460,-857.573242
346132,-550.844604,-898.714355,-763.393127


In [40]:
X4_pred1 = df4_pred1.iloc[:, 7:-1].values
# y4_pred1 = df4_pred1['theory'].values

X4_pred2 = df4_pred2.iloc[:, 7:-1].values
# # y4_pred2 = df4_pred2['theory'].values

# X4_pred3 = df4_pred3.iloc[:, 7:-1].values
# # y4_pred3 = df4_pred3['theory'].values

In [41]:
print(X4_pred1.shape)
print(X4_pred2.shape)
# print(X4_pred3.shape)

(27128, 3)
(27128, 3)


## Load model

In [ ]:
# 5 attributes
file = model_path+'model_nonzero_32to38_3layer5unit_300.sav'
model = pickle.load(open(file, 'rb'))

In [42]:
# 3 attributes
file = model_path+'best_model_nn_roi.h5'
model = load_model(file)

# Predict

In [43]:
df_test = df_test2
df_pred = df4_pred2

In [44]:
X_test = X4_pred2
# y_test = y4_pred3

In [ ]:
# GO TO Output section

In [ ]:
result = model.evaluate(X_test, y_test)
print(result)

In [ ]:
test_pred = model.predict(X_test)
print(mse(test_pred, y_test))
print(mape(test_pred, y_test))

In [ ]:
np.corrcoef(test_pred.flatten(), y_test)

# Plot

### Theory vs Predict

In [ ]:
x = y_test.flatten()
y = test_pred.flatten()
m, b, r, p, st_er = stats.linregress(x,y) 

yfit = [b + m * xi for xi in x]
yisx = [0 + 1 * xi for xi in x]
plt.plot(x, yfit)
plt.plot(x, yisx)

plt.scatter(y_test, test_pred,  color='black')
plt.axis([0,100, 0, 100])
plt.xlabel("Theory (nT)")
plt.ylabel("Prediction (nT)")
plt.title("Neural Network Prediction vs Theory", fontsize=15)
# print(r, st_er)
print("r: {:.5f}, st_er: {:.6f}".format(r, st_er))
print("y = "+str(round(m,4))+"*x + "+str(round(b,4)))

# Output data

In [45]:
test_pred = model.predict(X_test)

848/848 [==============================] - 1s 748us/step


In [46]:
df_pred['predict'] = test_pred

/var/folders/yd/vkjb_lqj2_30lnyhzhjqtg9w0000gp/T/ipykernel_7842/255584968.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pred['predict'] = test_pred


In [47]:
df_test['predict'] = 0.00

/var/folders/yd/vkjb_lqj2_30lnyhzhjqtg9w0000gp/T/ipykernel_7842/3257651465.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['predict'] = 0.00


In [160]:
# 5 attributes
df_out = df_test[['i', 'j', 'k', 'predict']]
df_pre = df_pred[['i', 'j', 'k', 'predict']]

In [48]:
# 3 attributes
df_out = df_test[['i', 'j', 'k', 'neg', 'predict']]
df_pre = df_pred[['i', 'j', 'k', 'neg', 'predict']]

In [49]:
df_test

,exp,mini_exp,i,j,k,amp,neg,mean0,mean1,mean2,target,predict
0,42,1,0,0,0,2,0,0.0,0.0,0.0,0.578396,0.0
1,42,1,0,0,1,2,0,0.0,0.0,0.0,0.584610,0.0
2,42,1,0,0,2,2,0,0.0,0.0,0.0,0.590888,0.0
3,42,1,0,0,3,2,0,0.0,0.0,0.0,0.597230,0.0
4,42,1,0,0,4,2,0,0.0,0.0,0.0,0.603638,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
180219,42,1,43,63,59,2,0,0.0,0.0,0.0,1.865481,0.0
180220,42,1,43,63,60,2,0,0.0,0.0,0.0,1.910840,0.0
180221,42,1,43,63,61,2,0,0.0,0.0,0.0,1.957465,0.0
180222,42,1,43,63,62,2,0,0.0,0.0,0.0,2.005388,0.0


In [162]:
# 5 attributes
for x in range(len(df_pre)):
    i = df_pre.iloc[x, :]['i'].astype(int)
    j = df_pre.iloc[x, :]['j'].astype(int)
    k = df_pre.iloc[x, :]['k'].astype(int)
    pred = df_pre.iloc[x, :]['predict']
    idx = df_out[(df_out['i']==i) & (df_out['j']==j) & (df_out['k']==k)].index
    df_out.loc[idx, 'predict']= pred

In [50]:
# 3 attributes
for x in range(len(df_pre)):
    i = df_pre.iloc[x, :]['i'].astype(int)
    j = df_pre.iloc[x, :]['j'].astype(int)
    k = df_pre.iloc[x, :]['k'].astype(int)
    neg = df_pre.iloc[x, :]['neg'].astype(int)
    pred = df_pre.iloc[x, :]['predict']
    idx = df_out[(df_out['i']==i) & (df_out['j']==j) & (df_out['k']==k) & (df_out['neg']==neg)].index
    df_out.loc[idx, 'predict']= pred

In [37]:
df_out

,i,j,k,neg,predict
0,0,0,0,0,0.0
1,0,0,1,0,0.0
2,0,0,2,0,0.0
3,0,0,3,0,0.0
4,0,0,4,0,0.0
...,...,...,...,...,...
180219,43,63,59,0,0.0
180220,43,63,60,0,0.0
180221,43,63,61,0,0.0
180222,43,63,62,0,0.0


In [164]:
# 5 attributes
np.savetxt(img_path+"nn_nonzero_neg_40_7_pos_erode_cly_2.txt", df_out[['i', 'j', 'k', 'predict']], fmt="%i %i %i %s")

In [116]:
# 3 attributes
df_pos = df_out[df_out['neg']==0]
df_neg = df_out[df_out['neg']==1]
np.savetxt(img_path+"nn_nonzero_neg_40_1_pos_erode_cly.txt", df_pos[['i', 'j', 'k', 'predict']], fmt="%i %i %i %s")
np.savetxt(img_path+"nn_nonzero_neg_40_1_neg_erode_cly.txt", df_neg[['i', 'j', 'k', 'predict']], fmt="%i %i %i %s")

In [51]:
# New 3 attributes
np.savetxt(img_path+"nn_nonzero_42_pos_roi_cly.txt", df_out[['i', 'j', 'k', 'predict']], fmt="%i %i %i %s")

In [42]:
df_out[df_out['neg']==0]

,i,j,k,neg,predict
0,0,0,0,0,0.0
1,0,0,1,0,0.0
2,0,0,2,0,0.0
3,0,0,3,0,0.0
4,0,0,4,0,0.0
...,...,...,...,...,...
180219,43,63,59,0,0.0
180220,43,63,60,0,0.0
180221,43,63,61,0,0.0
180222,43,63,62,0,0.0
